# 卷积神经网络 (Convolutional Neural Networks)

## 核心概念

卷积神经网络是一种专门用于处理具有网格结构数据的神经网络架构，特别适用于图像、序列等数据。

### 核心特性

1. **局部连接性** (Local Connectivity)
   - 每个神经元只与输入数据的局部区域连接
   - 大幅减少参数数量，提高计算效率
   - 符合视觉感受野的生物学特性

2. **参数共享** (Parameter Sharing)
   - 同一个卷积核在整个输入上滑动
   - 进一步减少参数，防止过拟合
   - 学习到的特征可在不同位置复用

3. **平移不变性** (Translation Invariance)
   - 对输入的空间位置变化具有鲁棒性
   - 特征检测不依赖于绝对位置
   - 提高模型的泛化能力

### 适用数据类型

- **Conv1D**: 一维序列数据（文本、时间序列、音频信号）
- **Conv2D**: 二维网格数据（灰度/彩色图像）
- **Conv3D**: 三维体积数据（视频、医学影像）

### 典型架构模式

```
Input → [Conv → Activation → Pooling] × N → Flatten/GlobalPooling → Dense → Output
```

---

## 1. 环境准备与数据加载

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt

print(f"TensorFlow版本: {tf.__version__}")
print(f"GPU可用: {tf.config.list_physical_devices('GPU')}")

# 设置随机种子以保证结果可复现
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# 加载CIFAR-10数据集
# CIFAR-10包含60000张32x32彩色图像，分为10个类别
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 数据集基本信息
print(f"训练集形状: {X_train.shape}")  # (50000, 32, 32, 3)
print(f"测试集形状: {X_test.shape}")    # (10000, 32, 32, 3)
print(f"标签形状: {y_train.shape}")     # (50000, 1)
print(f"数据类型: {X_train.dtype}")
print(f"像素值范围: [{X_train.min()}, {X_train.max()}]")

# 类别名称
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(class_names)

In [ ]:
# 数据可视化
plt.figure(figsize=(12, 6))
for i in range(20):
    plt.subplot(4, 5, i + 1)
    plt.imshow(X_train[i])
    plt.title(class_names[y_train[i][0]], fontsize=9)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# 数据预处理
# 1. 归一化到[0, 1]区间，加速收敛
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# 2. 标签one-hot编码
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(f"预处理后数据范围: [{X_train.min():.2f}, {X_train.max():.2f}]")
print(f"标签形状: {y_train.shape}")  # (50000, 10)

---

## 2. 标准卷积神经网络 (Standard CNN)

In [ ]:
def build_standard_cnn(input_shape=(32, 32, 3), num_classes=10):
    """
    构建标准卷积神经网络
    
    架构特点:
    - 逐层增加通道数(32→64→128)
    - 使用MaxPooling降低空间维度
    - 使用Dropout防止过拟合
    - 最后使用全局平均池化替代Flatten，减少参数
    
    参数:
        input_shape: 输入数据形状 (height, width, channels)
        num_classes: 分类类别数
    
    返回:
        编译后的Keras模型
    """
    model = models.Sequential(name='Standard_CNN')
    
    # 第一个卷积块
    # Conv2D(filters, kernel_size, ...)
    # - filters: 卷积核数量，决定输出通道数
    # - kernel_size: 卷积核大小，通常使用3x3或5x5
    # - padding='same': 保持空间维度不变
    model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                            padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())  # 批归一化，加速训练
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))  # 空间维度减半: 32x32 → 16x16
    model.add(layers.Dropout(0.25))         # 随机丢弃25%神经元
    
    # 第二个卷积块
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))  # 16x16 → 8x8
    model.add(layers.Dropout(0.25))
    
    # 第三个卷积块
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))  # 8x8 → 4x4
    model.add(layers.Dropout(0.25))
    
    # 全局平均池化层
    # 相比Flatten，GAP可以:
    # 1. 大幅减少参数 (4x4x128=2048 → 128)
    # 2. 降低过拟合风险
    # 3. 对输入尺寸更加灵活
    model.add(layers.GlobalAveragePooling2D())
    
    # 全连接分类层
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

In [ ]:
# 构建并编译模型
model_standard = build_standard_cnn()

# 编译模型
model_standard.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 查看模型结构
model_standard.summary()

In [ ]:
# 训练模型（使用简化参数进行测试）
history_standard = model_standard.fit(
    X_train, y_train,
    batch_size=128,
    epochs=50,  # 测试时使用2个epoch，实际训练建议50-100
    validation_split=0.1,
    verbose=1
)

---

## 3. 深度可分离卷积网络 (Depthwise Separable Convolution)

### 原理说明

深度可分离卷积将标准卷积分解为两步：

1. **Depthwise卷积**: 对每个输入通道独立进行空间卷积
2. **Pointwise卷积**: 1x1卷积对通道进行线性组合

### 计算复杂度对比

假设输入: `(H, W, C_in)`, 输出: `(H, W, C_out)`, 卷积核: `K×K`

- **标准卷积**: `H × W × C_in × C_out × K²` 次乘法
- **深度可分离卷积**: `H × W × C_in × K² + H × W × C_in × C_out` 次乘法

**计算量降低**: 约 `1/C_out + 1/K²` 倍

对于 `C_out=256, K=3`: 降低约 **8-9倍**

### 优势
- 参数量显著减少（约8-10倍）
- 训练速度更快
- 降低过拟合风险
- 准确率通常相当或略优于标准卷积

### 应用
- MobileNet系列（移动端模型）
- Xception网络
- EfficientNet系列

In [ ]:
def build_separable_cnn(input_shape=(32, 32, 3), num_classes=10):
    """
    构建深度可分离卷积网络
    
    使用SeparableConv2D替代Conv2D，显著减少参数量和计算量
    """
    model = models.Sequential(name='Separable_CNN')
    
    # 第一层仍使用标准卷积（输入通道较少时）
    model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                            padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    
    # 后续使用深度可分离卷积
    model.add(layers.SeparableConv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.SeparableConv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.SeparableConv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.SeparableConv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # 全局平均池化 + 分类层
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

In [ ]:
# 构建深度可分离卷积模型
model_separable = build_separable_cnn()

model_separable.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_separable.summary()

In [ ]:
# 参数量对比
standard_params = model_standard.count_params()
separable_params = model_separable.count_params()

print(f"标准卷积网络参数量: {standard_params:,}")
print(f"深度可分离卷积参数量: {separable_params:,}")
print(f"参数减少比例: {(1 - separable_params/standard_params)*100:.1f}%")

In [ ]:
# 训练深度可分离卷积模型
history_separable = model_separable.fit(
    X_train, y_train,
    batch_size=128,
    epochs=50,  # 测试时使用2个epoch
    validation_split=0.1,
    verbose=1
)

---

## 4. 模型评估与对比

In [ ]:
# 在测试集上评估
print("="*50)
print("标准卷积网络:")
loss_std, acc_std = model_standard.evaluate(X_test, y_test, verbose=0)
print(f"测试损失: {loss_std:.4f}")
print(f"测试精度: {acc_std:.4f}")

print("\n" + "="*50)
print("深度可分离卷积网络:")
loss_sep, acc_sep = model_separable.evaluate(X_test, y_test, verbose=0)
print(f"测试损失: {loss_sep:.4f}")
print(f"测试精度: {acc_sep:.4f}")

print("\n" + "="*50)
print("对比总结:")
print(f"参数量减少: {(1 - separable_params/standard_params)*100:.1f}%")
print(f"精度差异: {(acc_sep - acc_std)*100:+.2f}%")

In [ ]:
# 可视化训练历史
def plot_training_history(history, title):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # 损失曲线
    axes[0].plot(history.history['loss'], label='训练损失')
    axes[0].plot(history.history['val_loss'], label='验证损失')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title(f'{title} - 损失')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 精度曲线
    axes[1].plot(history.history['accuracy'], label='训练精度')
    axes[1].plot(history.history['val_accuracy'], label='验证精度')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title(f'{title} - 精度')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history_standard, '标准卷积网络')
plot_training_history(history_separable, '深度可分离卷积网络')

---

## 5. 预测与可视化

In [ ]:
# 随机选择一些测试样本进行预测
num_samples = 10
indices = np.random.choice(len(X_test), num_samples, replace=False)

# 获取预测结果
predictions = model_separable.predict(X_test[indices])
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test[indices], axis=1)

# 可视化预测结果
plt.figure(figsize=(15, 6))
for i in range(num_samples):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_test[indices[i]])
    
    pred_class = predicted_classes[i]
    true_class = true_classes[i]
    confidence = predictions[i][pred_class]
    
    color = 'green' if pred_class == true_class else 'red'
    title = f'预测: {class_names[pred_class]}\n'
    title += f'真实: {class_names[true_class]}\n'
    title += f'置信度: {confidence:.2%}'
    
    plt.title(title, fontsize=8, color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()

---

## 6. 关键知识总结

### 卷积层参数计算

对于 `Conv2D(filters, (k, k))`，如果输入通道数为 `C_in`:
- **参数量** = `k × k × C_in × filters + filters` (权重 + 偏置)
- **输出形状** = `(H_out, W_out, filters)`

其中:
- `H_out = (H_in + 2×padding - k) / stride + 1`
- `W_out = (W_in + 2×padding - k) / stride + 1`

### Pooling层作用

1. **降低空间维度**: 减少计算量和参数
2. **增强平移不变性**: 对微小位移更鲁棒
3. **扩大感受野**: 每个神经元看到更大的输入区域
4. **防止过拟合**: 丢弃部分空间信息

### 常用技巧

1. **批归一化** (Batch Normalization)
   - 加速训练收敛
   - 允许使用更大的学习率
   - 起到正则化作用

2. **Dropout**
   - 训练时随机丢弃神经元
   - 防止过拟合
   - 通常在全连接层使用较大比例(0.5)，卷积层使用较小比例(0.25)

3. **全局平均池化** vs **Flatten**
   - GAP参数更少，更不容易过拟合
   - GAP对输入尺寸变化更鲁棒
   - Flatten保留更多空间信息

### 网络设计原则

1. **逐层增加通道数**: 32 → 64 → 128 → 256
2. **逐层减小空间维度**: 通过pooling或stride=2的卷积
3. **深度优先**: 多个小卷积核(3×3)优于单个大卷积核(5×5)
4. **正则化**: BN + Dropout + 数据增强
5. **激活函数**: ReLU及其变体(LeakyReLU, ELU)表现最好

---

## 完整训练代码模板

```python
# 实际项目中建议使用的完整训练配置
# epochs = 100
# batch_size = 64

# callbacks = [
#     keras.callbacks.EarlyStopping(
#         monitor='val_loss',
#         patience=10,
#         restore_best_weights=True
#     ),
#     keras.callbacks.ReduceLROnPlateau(
#         monitor='val_loss',
#         factor=0.5,
#         patience=5,
#         min_lr=1e-7
#     ),
#     keras.callbacks.ModelCheckpoint(
#         'best_model.h5',
#         monitor='val_accuracy',
#         save_best_only=True
#     )
# ]

# history = model.fit(
#     X_train, y_train,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=0.1,
#     callbacks=callbacks,
#     verbose=1
# )
```